# Data 1050 SQL Project

## Create Database

In [1]:
pip install mysql-connector-python # type: ignore

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector # type: ignore
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password= "7@u*sqJX"   #REPLACE THIS WITH THE PASSWORD YOU SET
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

CONNECTION SUCCESSFUL


In [3]:
#create a database
mycursor = mydb.cursor()
mycursor.execute("DROP DATABASE IF EXISTS data1050SQLProject")
mycursor.execute("CREATE DATABASE data1050SQLProject")

In [4]:
mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")

for x in mycursor:
  print(x)

('data1050f24',)
('data1050SQLProject',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


### Connecting to a database and showing tables

In [5]:
#connect to a specific database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password= "7@u*sqJX", #REPLACE THIS WITH YOUR PASSWORD 
  database = "data1050SQLProject" #connecting to testDatabase
)

In [6]:
mycursor = mydb.cursor()

mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

In [7]:
import pandas as pd # type: ignore

In [8]:
pip install fsspec # type: ignore

Note: you may need to restart the kernel to use updated packages.


### Add Tables

In [9]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.physicians")
mycursor.execute('''CREATE TABLE data1050SQLProject.physicians ( 
                    ssn VARCHAR(128) PRIMARY KEY, 
                    name VARCHAR(128), 
                    primary_specialty VARCHAR(128),
                    experience_years INT);''')

In [10]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.patients")
mycursor.execute('''CREATE TABLE data1050SQLProject.patients ( 
                    ssn VARCHAR(128) PRIMARY KEY, 
                    name VARCHAR(128), 
                    address VARCHAR(128),
                    birth_date VARCHAR(128),
                    physician_id VARCHAR(128),
                    foreign key(physician_id) references data1050SQLProject.physicians(ssn));''')

In [11]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.pharmacies")
mycursor.execute('''CREATE TABLE data1050SQLProject.pharmacies ( 
                    id INT PRIMARY KEY, 
                    name VARCHAR(128), 
                    address VARCHAR(128),
                    phone VARCHAR(128));''')

In [12]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.drugs")
mycursor.execute('''CREATE TABLE data1050SQLProject.drugs ( 
                    id VARCHAR(128), 
                    name VARCHAR(128) PRIMARY KEY);''')

In [13]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.prescriptions")
mycursor.execute('''CREATE TABLE data1050SQLProject.prescriptions ( 
                    id INT PRIMARY KEY, 
                    patient_id VARCHAR(128), 
                    physician_id VARCHAR(128),
                    drug_name VARCHAR(128),
                    date VARCHAR(128),
                    quantity INT,
                    unique(patient_id,drug_name),
                    foreign key(patient_id) references data1050SQLProject.patients(ssn),
                    foreign key(physician_id) references data1050SQLProject.physicians(ssn),
                    foreign key(drug_name) references data1050SQLProject.drugs(name));''')

In [14]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.adverse_interactions")
mycursor.execute('''CREATE TABLE data1050SQLProject.adverse_interactions ( 
                    drug_name VARCHAR(128), 
                    drug_name_2 VARCHAR(128), 
                    PRIMARY KEY (drug_name,drug_name_2),
                    foreign key(drug_name) references data1050SQLProject.drugs(name));''')

In [15]:
#create a table
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.alerts")
mycursor.execute('''CREATE TABLE data1050SQLProject.alerts ( 
                    patient_id VARCHAR(128), 
                    physician_id VARCHAR(128), 
                    alert_date DATETIME,
                    drug1 VARCHAR(128),
                    drug2 VARCHAR(128),
                    PRIMARY KEY (patient_id,physician_id,alert_date,drug1,drug2),
                    foreign key(patient_id) references data1050SQLProject.patients(ssn),
                    foreign key(physician_id) references data1050SQLProject.physicians(ssn),
                    foreign key(patient_id,drug1) references data1050SQLProject.prescriptions(patient_id,drug_name),
                    foreign key(patient_id,drug2) references data1050SQLProject.prescriptions(patient_id,drug_name)
                 );''')

In [16]:
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.pharmacy_fills")
mycursor.execute('''CREATE TABLE data1050SQLProject.pharmacy_fills ( 
                    prescription_id INT, 
                    pharmacy_id INT, 
                    date VARCHAR(128),
                    cost DECIMAL(5,2),
                    PRIMARY KEY (prescription_id,pharmacy_id),
                    foreign key(prescription_id) references data1050SQLProject.prescriptions(id),
                    foreign key(pharmacy_id) references data1050SQLProject.pharmacies(id));''')

In [17]:
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.companies")
mycursor.execute('''CREATE TABLE data1050SQLProject.companies ( 
                    id INT PRIMARY KEY, 
                    name VARCHAR(128), 
                    address VARCHAR(128),
                    contact_phone VARCHAR(128),
                    contact_name VARCHAR(128));''')

In [18]:
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS data1050SQLProject.contracts")
mycursor.execute('''CREATE TABLE data1050SQLProject.contracts ( 
                    id INT PRIMARY KEY, 
                    drug_name VARCHAR(128),
                    dosage INT,
                    pharmacy_id INT,
                    company_id INT,
                    quantity INT,
                    date VARCHAR(128),
                    price INT,
                    foreign key(company_id) references data1050SQLProject.companies(id),
                    foreign key(pharmacy_id) references data1050SQLProject.pharmacies(id),
                    foreign key(drug_name) references data1050SQLProject.drugs(name));''')

In [19]:
mycursor = mydb.cursor()

mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

('adverse_interactions',)
('alerts',)
('companies',)
('contracts',)
('drugs',)
('patients',)
('pharmacies',)
('pharmacy_fills',)
('physicians',)
('prescriptions',)


### Add Data to Tables

In [20]:
#point the path to where in your hard drive you have stored the file physicians.csv
physicians_df = pd.read_csv("physicians.csv")
physicians_df.head()

,SSN,name,primary_specialty,experience_years
0,614-57-6885,Srinivasan,Cardiology,4
1,702-16-8749,Wu,Dermatology,10
2,571-13-9020,Mozart,Cardiology,0
3,718-27-0905,Einstein,Psychiatry,29
4,230-12-3219,El Said,Psychiatry,12


In [21]:
physicians_df.dtypes

SSN                   object
name                  object
primary_specialty     object
experience_years       int64
dtype: object

In [22]:
for i,row in physicians_df.iterrows():
            sql = "INSERT INTO physicians VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [23]:
query = ''' SELECT * from physicians'''
mycursor.execute(query)

In [24]:
for x in mycursor:
    print(x)

('118-66-5958', 'Katz       ', 'Orthopedics       ', 3)
('156-28-1945', 'Singh      ', 'Orthopedics       ', 25)
('163-50-5535', 'Gold       ', 'Neurology         ', 8)
('230-12-3219', 'El Said    ', 'Psychiatry        ', 12)
('357-93-5814', 'Califieri  ', 'Cardiology        ', 21)
('460-35-6754', 'Kim        ', 'Orthopedics       ', 2)
('510-55-9776', 'Brandt     ', 'Psychiatry        ', 25)
('522-86-5827', 'Crick      ', 'Neurology         ', 0)
('571-13-9020', 'Mozart     ', 'Cardiology        ', 0)
('614-57-6885', 'Srinivasan ', 'Cardiology        ', 4)
('702-16-8749', 'Wu         ', 'Dermatology       ', 10)
('718-27-0905', 'Einstein   ', 'Psychiatry        ', 29)


In [25]:
#point the path to where in your hard drive you have stored the file patients.csv
data = pd.read_csv("patients.csv")
data.head()

,ssn,name,address,birthdate,physician_id
0,478-34-0781,Florance Saiz,7 Fair Oaks Place,1988-11-03T23:25:38Z,614-57-6885
1,885-94-4721,Merry Di Pietro,1 Old Shore Court,1991-02-07T22:00:41Z,702-16-8749
2,777-39-3296,Myron Cottem,75875 Fulton Crossing,1986-02-20T04:43:29Z,718-27-0905
3,227-08-7452,Bearnard Remer,18669 Heffernan Point,2008-01-09T05:34:30Z,230-12-3219
4,805-15-2755,Roxana Worster,54 Hudson Junction,1982-11-12T18:11:55Z,163-50-5535


In [26]:
data.dtypes

ssn                         object
name                        object
address                     object
birthdate                   object
physician_id                object
dtype: object

In [27]:
query = "SELECT * FROM physicians WHERE ssn = '614-57-6885';"
mycursor.execute(query)


In [28]:
for x in mycursor:
    print(x)

('614-57-6885', 'Srinivasan ', 'Cardiology        ', 4)


In [29]:
for i,row in data.iterrows():
            sql = "INSERT INTO patients VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [30]:
query = "SELECT * from patients"
mycursor.execute(query)

In [31]:
for x in mycursor:
    print(x)

('192-33-2887 ', 'Jacinda Stowe      ', '8 Colorado Alley        ', '1970-04-15T00:24:26Z ', '357-93-5814')
('227-08-7452 ', 'Bearnard Remer     ', '18669 Heffernan Point   ', '2008-01-09T05:34:30Z ', '230-12-3219')
('303-13-5928 ', 'Krystyna Luckie    ', '54106 Barnett Plaza     ', '1950-02-11T12:20:13Z ', '571-13-9020')
('360-47-2098 ', 'Peter Lukasen      ', '552 Ryan Court          ', '1969-01-10T19:33:03Z ', '522-86-5827')
('478-34-0781 ', 'Florance Saiz      ', '7 Fair Oaks Place       ', '1988-11-03T23:25:38Z ', '614-57-6885')
('501-47-2038 ', 'Elvyn Rudinger     ', '48 Bowman Parkway       ', '2006-02-28T16:26:43Z ', '156-28-1945')
('631-75-6048 ', 'Avrom Messer       ', '5030 Garrison Center    ', '1929-02-04T06:34:10Z ', '510-55-9776')
('691-21-7304 ', 'Myrlene Yegoshin   ', '2 Sunnyside Court       ', '2001-06-03T23:02:52Z ', '460-35-6754')
('758-08-7274 ', 'Susanetta Petruska ', '16276 Sutteridge Avenue ', '1922-08-05T18:36:12Z ', '118-66-5958')
('777-39-3296 ', 'Myron Cott

In [32]:
#point the path to where in your hard drive you have stored the file pharmacies.csv
data = pd.read_csv("pharmacies.csv")
data.head()

,id,name,address,phone
0,1,Springfield Pharmacy,"123 Main St, Springfield, IL 62701",(217) 555-1234
1,2,Peachtree Meds,"456 Elm St, Atlanta, GA 30303",(404) 555-5678
2,3,Lone Star Drugs,"789 Oak St, Dallas, TX 75201",(214) 555-9101
3,4,Mile High Meds,"101 Pine St, Denver, CO80202",(303) 555-1121
4,5,Emerald City Pharmacy,"121 Spruce St, Seattle, WA 98101",(206) 555-1314


In [33]:
data.dtypes

id                                             int64
name                                          object
address                                       object
phone                                         object
dtype: object

In [34]:
for i,row in data.iterrows():
            sql = "INSERT INTO pharmacies VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [35]:
query = " SELECT * FROM pharmacies"
mycursor.execute(query)

In [36]:
for x in mycursor:
    print(x)

(1, 'Springfield Pharmacy  ', ' 123 Main St,  Springfield,  IL 62701   ', '(217) 555-1234')
(2, 'Peachtree Meds        ', ' 456 Elm St,  Atlanta, GA 30303         ', '(404) 555-5678')
(3, 'Lone Star Drugs       ', ' 789 Oak St,  Dallas, TX 75201          ', '(214) 555-9101')
(4, 'Mile High Meds        ', ' 101 Pine St, Denver, CO80202           ', '(303) 555-1121')
(5, 'Emerald City Pharmacy ', ' 121 Spruce St,  Seattle,  WA 98101     ', '(206) 555-1314')
(6, 'Golden Gate Drugs     ', ' 234 Market St, San Francisco, CA 94105 ', '(415) 555-1515')
(7, 'Sunshine Pharmacy     ', ' 345 Palm Ave, Miami, FL 33101          ', '(305) 555-1616')
(8, 'Liberty Meds          ', ' 567 Broadway St, New York, NY 10001    ', '(212) 555-1717')
(9, 'Lakeside Drugs        ', ' 678 Lake Rd,  Minneapolis, MN 55401    ', '(612) 555-1818')
(10, 'Desert Bloom Pharmacy ', ' 890 Desert Blvd, Phoenix, AZ 85001     ', '(602) 555-1919')
(11, 'Bayside Pharmacy      ', ' 112 Harbor Dr, San Diego, CA 92101     ', '(61

In [37]:
#point the path to where in your hard drive you have stored the file drugs.csv
data = pd.read_csv("drugs.csv")
data.head()

,drug_id,drug_name
0,1,Primalovir
1,2,Olanzanafine
2,3,Avafoxin
3,4,Quixiposide
4,5,Cleotrana


In [38]:
data.dtypes

drug_id       int64
drug_name    object
dtype: object

In [39]:
for i,row in data.iterrows():
            sql = "INSERT INTO drugs VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [40]:
query = "SELECT * FROM drugs"
mycursor.execute(query)

In [41]:
for x in mycursor:
    print(x)

('9', 'Abnazole Toleluble')
('3', 'Avafoxin')
('5', 'Cleotrana')
('10', 'Dantopex Quixilinum')
('8', 'Divisporine Acetaclotide')
('7', 'Glucozepam Amcipentin')
('6', 'Kanulin')
('2', 'Olanzanafine')
('1', 'Primalovir')
('4', 'Quixiposide')


In [42]:
#point the path to where in your hard drive you have stored the file prescriptions.csv
data = pd.read_csv("prescriptions.csv")
data.head()

,id,patient_id,physician_id,drug_name,date,quantity
0,1,478-34-0781,614-57-6885,Avafoxin,3/11/2023,90
1,2,758-08-7274,118-66-5958,Cleotrana,3/12/2023,10
2,3,758-08-7274,118-66-5958,Primalovir,4/11/2023,20
3,4,758-08-7274,118-66-5958,Glucozepam Amcipentin,5/13/2023,12
4,5,303-13-5928,571-13-9020,Olanzanafine,5/24/2023,25


In [43]:
data.dtypes

id                int64
patient_id       object
physician_id     object
drug_name        object
date             object
quantity          int64
dtype: object

In [44]:
for i,row in data.iterrows():
            sql = "INSERT INTO prescriptions VALUES (%s,%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


IntegrityError: 1062 (23000): Duplicate entry '478-34-0781 -Avafoxin' for key 'prescriptions.patient_id'

In [43]:
query = "SELECT * FROM prescriptions"
mycursor.execute(query)

In [44]:
for x in mycursor:
    print(x)

In [45]:
#point the path to where in your hard drive you have stored the file prescriptions.csv
data = pd.read_csv("prescriptions.csv")
data.head()

,id,patient_id,physician_id,drug_name,date,quantity
0,1,478-34-0781,614-57-6885,Avafoxin,3/11/2023,90
1,2,758-08-7274,118-66-5958,Cleotrana,3/12/2023,10
2,3,758-08-7274,118-66-5958,Primalovir,4/11/2023,20
3,4,758-08-7274,118-66-5958,Glucozepam Amcipentin,5/13/2023,12
4,5,303-13-5928,571-13-9020,Olanzanafine,5/24/2023,25


In [46]:
#point the path to where in your hard drive you have stored the file adverse_interactions.csv
data = pd.read_csv("adverse_reactions.csv")
data.head()

,drug_name_1,drug_name_2
0,Cleotrana,Kanulin
1,Primalovir,Abnazole Toleluble
2,Primalovir,Olanzanafine
3,Olanzanafine,Glucozepam Amcipentin
4,Avafoxin,Kanulin


In [47]:
data.dtypes

drug_name_1      object
drug_name_2      object
dtype: object

In [ ]:
# for i,row in data.iterrows():
#             sql = "INSERT INTO adverse_interactions VALUES (%s,%s)"
#             mycursor.execute(sql, tuple(row))
#             print("Record inserted")
#             # the connection is not autocommitted by default, so we 
#             # must commit to save our changes
#             mydb.commit()

In [49]:
# query = "SELECT * FROM adverse_interactions"
# mycursor.execute(query)

In [50]:
# for x in mycursor:
#     print(x)

In [51]:
#point the path to where in your hard drive you have stored the file pharmacy_fills.csv
data = pd.read_csv("pharmacy_fills.csv")
data.head()

,pharmacy_id,prescription_id,date,cost
0,1,3,3/15/2023,60.53
1,3,4,5/16/2023,41.50
2,1,2,3/12/2023,18.00
3,8,1,3/12/2023,46.53
4,10,5,5/26/2023,47.50


In [52]:
data.dtypes

pharmacy_id           int64
prescription_id       int64
date                 object
cost                float64
dtype: object

In [53]:
# for i,row in data.iterrows():
#             sql = "INSERT INTO pharmacy_fills VALUES (%s,%s,%s,%s)"
#             mycursor.execute(sql, tuple(row))
#             print("Record inserted")
#             # the connection is not autocommitted by default, so we 
#             # must commit to save our changes
#             mydb.commit()

In [54]:
# query = "SELECT * FROM pharmacy_fills"
# mycursor.execute(query)

In [55]:
# for x in mycursor:
#     print(x)

In [56]:
#point the path to where in your hard drive you have stored the file companies.csv
data = pd.read_csv("companies.csv")
data.head()

,id,name,address,contact_phone,contact_name
0,1,Goodrx,"123 Main St, San Francisco, CA",123-456-7890,Holly Jolly
1,2,PHARMASEE,"456 Elm St, New York, NY",234-567-8901,Faker Maker
2,3,DRUGXO,"789 Maple St, Los Angeles, CA",345-678-9012,Silly Putty
3,4,Pharmachoice,"101 Pine St, Chicago, IL",456-789-0123,Connie Honey
4,5,Castox,"234 Oak St, Houston, TX",567-890-1234,Laxmi Kant Sheth


In [57]:
data.dtypes

id                                    int64
name                                 object
address                              object
contact_phone                        object
contact_name                         object
dtype: object

In [58]:
for i,row in data.iterrows():
            sql = "INSERT INTO companies VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [59]:
query = "SELECT * FROM companies"
mycursor.execute(query)

In [60]:
for x in mycursor:
    print(x)

(1, 'Goodrx       ', '123 Main St, San Francisco, CA ', '123-456-7890  ', 'Holly Jolly')
(2, 'PHARMASEE    ', '456 Elm St, New York, NY       ', '234-567-8901  ', 'Faker Maker')
(3, 'DRUGXO       ', '789 Maple St, Los Angeles, CA  ', '345-678-9012  ', 'Silly Putty')
(4, 'Pharmachoice ', '101 Pine St, Chicago, IL       ', '456-789-0123  ', 'Connie Honey')
(5, 'Castox       ', '234 Oak St, Houston, TX        ', '567-890-1234  ', 'Laxmi Kant Sheth')
(6, 'Doktera      ', '567 Cedar St, Philadelphia, PA ', '678-901-2345  ', 'I.P. Green')
(7, 'Lipdrugz     ', '890 Birch St, Phoenix, AZ      ', '789-012-3456  ', 'Boris Kotchakoff')
(8, 'Nurfarma     ', '123 Fir St, San Antonio, TX    ', '890-123-4567  ', 'Wu Liu')
(9, 'Munimed      ', '456 Redwood St, San Diego, CA  ', '901-234-5678  ', 'Kim Park')
(10, 'Arkmed       ', '789 Sequoia St, Dallas, TX     ', '012-345-6789  ', 'James Bond')


In [61]:
#point the path to where in your hard drive you have stored the file contracts.csv
data = pd.read_csv("contracts.csv")
data.head()

,Contract_Id,drug,dosage,pharmacy_id,pharm_company_id,quantity,date,price
0,1,Cleotrana,50,5,10,40,10/3/2023,100.0
1,2,Primalovir,500,5,10,20,9/26/2023,40.5
2,3,Kanulin,1000,4,9,20,9/20/2023,10.5
3,4,Olanzanafine,50,6,3,80,9/24/2023,150.0
4,5,Avafoxin,5,15,5,30,9/29/2023,18.0


In [62]:
data.dtypes

Contract_Id                    int64
drug                          object
dosage                         int64
pharmacy_id                    int64
pharm_company_id               int64
quantity                       int64
date                          object
price                        float64
dtype: object

In [63]:
for i,row in data.iterrows():
            sql = "INSERT INTO contracts VALUES (%s,%s,%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

ProgrammingError: Not all parameters were used in the SQL statement

In [ ]:
query = "SELECT * FROM contracts"
mycursor.execute(query)

In [ ]:
for x in mycursor:
    print(x)